In [ ]:
import os
import re
import time
import glob
import shutil
import threading

import pandas as pd
from tqdm import tqdm
import pyautogui
import xlwings as xw
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment
from openpyxl.utils import get_column_letter


# 📦 Definitions

## linha_vazia

In [ ]:
def linha_vazia(ws, row_index):
    if ws.row_dimensions[row_index].hidden:
        for c in range(1, ws.max_column + 1):
            ws.cell(row=row_index, column=c).value = None
        return True
    for c in range(1, ws.max_column + 1):
        val = ws.cell(row=row_index, column=c).value
        if val not in [None, 0, -0, 0.0, -0.0]:
            return False
    return True


## coluna_vazia

In [ ]:
def coluna_vazia(ws, col_index):
    if ws.column_dimensions[get_column_letter(col_index)].hidden:
        return True
    for r in range(1, ws.max_row + 1):
        val = ws.cell(row=r, column=col_index).value
        if val not in [None, 0, -0, 0.0, -0.0]:
            return False
    return True

## processar_arquivo

In [ ]:
def processar_arquivo(caminho_arquivo):
    try:
        wb = load_workbook(caminho_arquivo)
        for nome_sheet in wb.sheetnames:
            sheet = wb[nome_sheet]
            linha_vazia_count = 0
            for r in range(sheet.max_row, 0, -1):
                if linha_vazia(sheet, r):
                    linha_vazia_count += 1
                    if linha_vazia_count > 2:
                        sheet.delete_rows(r)
                else:
                    linha_vazia_count = 0

            coluna_vazia_count = 0
            for c in range(sheet.max_column, 0, -1):
                if coluna_vazia(sheet, c):
                    coluna_vazia_count += 1
                    if coluna_vazia_count > 2:
                        sheet.delete_cols(c)
                else:
                    coluna_vazia_count = 0

            for row in range(1, sheet.max_row + 1):
                sheet.row_dimensions[row].hidden = False
            for col in range(1, sheet.max_column + 1):
                sheet.column_dimensions[get_column_letter(col)].hidden = False

        wb.save(caminho_arquivo)
    except Exception as e:
        print(f"Erro ao processar o arquivo {caminho_arquivo}: {e}")

## verificar_e_mover_arquivo

In [ ]:
def verificar_e_mover_arquivo(arquivo, diretorio_invalido):
    # Nome do arquivo sem a extensão
    nome_arquivo = os.path.splitext(os.path.basename(arquivo))[0][1:].lstrip('0')

    # Leia o arquivo xlsx
    df = pd.read_excel(arquivo, header=None)
    
    if df.iloc[1, 2] != nome_arquivo:
        # Crie o diretório se não existir
        if not os.path.exists(diretorio_invalido):
            os.makedirs(diretorio_invalido)
        # Mova o arquivo para o diretório inválido
        shutil.move(arquivo, os.path.join(diretorio_invalido, os.path.basename(arquivo)))

## salve_file

In [ ]:
def salve_file(input_file,template_file= "template.xlsx", sheet_name="IR_CS_ANUAL"):
    output_file = f"testes2/{os.path.basename(input_file)}"
    if os.path.isfile(output_file):
        return
    # Passo 1: Criar uma cópia do template
    shutil.copy(template_file, output_file)

    # Abrir o arquivo de entrada e o arquivo de saída
    wb_input = openpyxl.load_workbook(input_file,read_only=True, data_only=True)
    wb_output = openpyxl.load_workbook(output_file)

    # Verificar se a aba IR_CS_ANUAL existe no arquivo de entrada
    if sheet_name not in wb_input.sheetnames:
        raise ValueError(f"A aba {sheet_name} não existe no arquivo de entrada.")

    # Selecionar a aba IR_CS_ANUAL do arquivo de entrada e de saída
    ws_input = wb_input[sheet_name]
    ws_output = wb_output.active
    ws_output.title = sheet_name

    # Copiar apenas os valores para a aba do template
    # max_row = 0
    # max_col = 0
    for row_idx, row in enumerate(ws_input.iter_rows(), start=1):
        # is_row_filled = False
        for col_idx, cell in enumerate(row, start=1):
            new_cell = ws_output.cell(row=row_idx, column=col_idx)
            new_cell.value = cell.value  # Copia apenas o valor da célula, sem fórmulas

            # Formatação condicional
            if isinstance(cell.value, str):
                if re.match(r"^\d{8}\s", cell.value):  # 8 números seguidos de espaço
                    new_cell.font = Font(bold=True)
                elif re.match(r"^\d{9,}", cell.value):  # Mais de 8 números no início
                    new_cell.font = Font(bold=False)
                    new_cell.alignment = Alignment(indent=1)

        #     if cell.value is not None:
        #         is_row_filled = True
        #         max_col = max(max_col, col_idx)

        # if is_row_filled:
        #     max_row = row_idx

    

    # Remover linhas após a última linha com valores
    # for row_idx in range(max_row + 1, ws_output.max_row + 1):
    #     ws_output.delete_rows(max_row + 1)

    # # Remover colunas após a última coluna com valores
    # for col_idx in range(max_col + 1, ws_output.max_column + 1):
    #     ws_output.delete_cols(max_col + 1)

    # # Remover linhas vazias entre blocos de texto
    # row_idx = 1
    # while row_idx <= max_row:
    #     if is_row_empty(ws_output, row_idx):
    #         # Verifica se a próxima linha também está vazia
    #         if is_row_empty(ws_output, row_idx + 1):
    #             ws_output.delete_rows(row_idx)
    #             max_row -= 1
    #         else:
    #             row_idx += 1
    #     else:
    #         row_idx += 1

    # Salvar o arquivo modificado
    wb_output.save(output_file)
    wb_input.close()
    wb_output.close()

    # print(f"Arquivo salvo como {output_file}")


## processar_arquivos_com_threads

In [ ]:

def processar_arquivos_com_threads(diretorio: str, diretorio_invalido: str, funcao_verificacao, thread_limit: int = 60):
    """
    Processa arquivos em um diretório usando múltiplas threads, aplicando uma função de verificação em cada arquivo.
    
    Parâmetros:
    - diretorio: Caminho onde estão os arquivos a serem processados.
    - diretorio_invalido: Caminho para mover arquivos considerados inválidos.
    - funcao_verificacao: Função a ser executada em cada arquivo. Deve receber (caminho_arquivo, diretorio_invalido).
    - thread_limit: Número máximo de threads simultâneas. Padrão é 60.
    """
    os.makedirs(diretorio_invalido, exist_ok=True)
    arquivos = os.listdir(diretorio)

    threads = []
    for idx, arquivo in enumerate(tqdm(arquivos, desc="Processando arquivos", total=len(arquivos))):
        caminho_arquivo = os.path.join(diretorio, arquivo)
        thread = threading.Thread(target=funcao_verificacao, args=(caminho_arquivo, diretorio_invalido))
        thread.start()
        threads.append(thread)

        if len(threads) >= thread_limit or idx == len(arquivos) - 1:
            for thread in threads:
                thread.join()
            threads = []

    print("Verificação concluída.")


# ▶️ Execução

## Remove . dos nomes de arquivos

In [ ]:
diretorio = 'C:/Users/patricklima/KPMG/Rufino, Jaqueline S - Contratos separados'

In [ ]:
arquivos = os.listdir(diretorio)

# Itere sobre cada arquivo
for arquivo in arquivos:
    # Verifique se o arquivo contém um ponto
    if '.' in arquivo:
        # Encontre a posição do primeiro ponto
        pos_ponto = arquivo.find('.')
        # Remova o primeiro ponto do nome do arquivo
        novo_nome = arquivo[:pos_ponto] + arquivo[pos_ponto+1:]
        try:
            # Renomeie o arquivo
            os.rename(os.path.join(diretorio, arquivo), os.path.join(diretorio, novo_nome))
        except PermissionError:
            print(f"Permissão negada: {arquivo}")

print("Renomeação concluída.")

## Verifica Erro em C

In [ ]:
processar_arquivos_com_threads('testes2', 'invalido', verificar_e_mover_arquivo)


## Pega lista de ERROS em C

In [ ]:
# Liste todos os arquivos no diretório inválido
diretorio_invalido = 'invalido'
arquivos_invalidos = os.listdir(diretorio_invalido)

# Crie um DataFrame com os nomes dos arquivos
df_invalidos = pd.DataFrame(arquivos_invalidos, columns=['documento'])

# Salve o DataFrame em um arquivo CSV
df_invalidos.to_csv('arquivos_invalidos.csv', index=False)

print("Arquivo CSV criado com sucesso.")

## Renomeia C para adicionar zeros

In [ ]:
diretorio = 'testes2'
arquivos = os.listdir(diretorio)
# Itere sobre cada arquivo no diretório
for arquivo in arquivos:
    # Verifique se o arquivo contém um ponto
    if '.' in arquivo:
        # Encontre a posição do primeiro ponto
        pos_ponto = arquivo.find('.')
        # Separe o nome do arquivo e a extensão
        nome_arquivo = arquivo[:pos_ponto]
        extensao = arquivo[pos_ponto:]
        # Separe a letra 'C' e os números
        letra = nome_arquivo[0]
        numeros = nome_arquivo[1:]
        # Complete com zeros à esquerda se necessário
        numeros_completos = numeros.zfill(7)
        # Crie o novo nome do arquivo
        novo_nome = letra + numeros_completos + extensao
        # Renomeie o arquivo
        try:
            os.rename(os.path.join(diretorio, arquivo), os.path.join(diretorio, novo_nome))
        except:
            pass
print("Renomeação concluída.")

## Renomeia B para adicionar zeros

In [ ]:
# Liste todos os arquivos no diretório B
arquivos = os.listdir('B')

# Itere sobre cada arquivo no diretório
for arquivo in arquivos:
    # Verifique se o arquivo começa com 'B' e contém um ponto
    if arquivo.startswith('B') and '.' in arquivo:
        # Encontre a posição do primeiro ponto
        pos_ponto = arquivo.find('.')
        # Separe o nome do arquivo e a extensão
        nome_arquivo = arquivo[:pos_ponto]
        extensao = arquivo[pos_ponto:]
        # Remova os zeros após 'B' e antes do primeiro número
        novo_nome_arquivo = 'B' + nome_arquivo[1:].lstrip('0')
        # Crie o novo nome do arquivo
        novo_nome = novo_nome_arquivo + extensao
        # Renomeie o arquivo
        os.rename(os.path.join('B', arquivo), os.path.join('B', novo_nome))

print("Renomeação concluída.")

## Fix lines and spaces

In [ ]:
diretorio = 'Input/manual'


In [ ]:
arquivos = [arquivo for arquivo in os.listdir(diretorio) if arquivo.lower().endswith('.xlsx')]

filtros_df = pd.read_excel('Output/filtro.xlsx')
filtros_df["Contrato"] = filtros_df["Contrato"].astype(str).str.zfill(7)
documentos = filtros_df["Contrato"].values.tolist()
arquivos = [ arquivo for arquivo in arquivos if re.sub(r'\D', '', arquivo) in documentos]

thread_limit = 30
threads = []

for idx, arquivo in tqdm(enumerate(arquivos), desc="Processando arquivos", total=len(arquivos)):
    # if idx<3653-30:
    #     continue
    caminho_arquivo = os.path.join(diretorio, arquivo)
    thread = threading.Thread(target=processar_arquivo, args=(caminho_arquivo,))
    thread.start()
    threads.append(thread)
    if len(threads) >= thread_limit or idx == len(arquivos) - 1:
        for thread in threads:
            thread.join()
        threads = []

print("Modificações concluídas.")


In [ ]:
import re
import os
import pandas as pd
diretorio = 'C:/Users/patricklima/Downloads/preparar_envio/Anexo_N_A'
arquivos = [arquivo for arquivo in os.listdir(diretorio) if arquivo.lower().endswith('.xlsx')]
filtros_df = pd.read_excel('filtro.xlsx')
filtros_df["Contrato"] = filtros_df["Contrato"].astype(str).str.zfill(7)
documentos = filtros_df["Contrato"].values.tolist()
arquivos = [ arquivo for arquivo in arquivos if re.sub(r'\D', '', arquivo) in documentos]
# Create the 'filtrado' directory if it doesn't exist
filtrado_dir = 'filtrado'
if not os.path.exists(filtrado_dir):
    os.makedirs(filtrado_dir)

# Copy the filtered files to the 'filtrado' directory
for arquivo in arquivos:
    src_path = os.path.join(diretorio, arquivo)
    dest_path = os.path.join(filtrado_dir, arquivo)
    if os.path.isfile(src_path):
        with open(src_path, 'rb') as src_file:
            with open(dest_path, 'wb') as dest_file:
                dest_file.write(src_file.read())

In [ ]:
diretorio = 'C:/Users/patricklima/Downloads/preparar_envio/Anexo_N_A/'
arquivos = [arquivo for arquivo in os.listdir(diretorio) if arquivo.lower().endswith('.pdf')]
filtros_df = pd.read_excel('filtro.xlsx')
filtros_df["Contrato"] = filtros_df["Contrato"].astype(str).str.zfill(7)
documentos = filtros_df["Contrato"].values.tolist()
arquivos = [re.sub(r'\D', '', arquivo)for arquivo in arquivos]
print(len(arquivos))
print(len([ arquivo for arquivo in documentos if re.sub(r'\D', '', arquivo) not in arquivos]))
print([ arquivo for arquivo in documentos if re.sub(r'\D', '', arquivo) not in arquivos])

## Mover

In [ ]:
import os
import shutil

# Defina o caminho do diretório
diretorio = 'B'

# Liste todos os arquivos no diretório
arquivos = os.listdir(diretorio)

# Itere sobre cada arquivo no diretório
for arquivo in arquivos:
    # Verifique se o arquivo contém um ponto
    if '.' in arquivo:
        # Encontre a posição do primeiro ponto
        pos_ponto = arquivo.find('.')
        # Separe o nome do arquivo e a extensão
        nome_arquivo = arquivo[:pos_ponto]
        extensao = arquivo[pos_ponto:]
        # Separe a letra e os números
        letra = nome_arquivo[0]
        numeros = nome_arquivo[1:]
        # Crie o nome do diretório com base nos números
        nome_diretorio = os.path.join(diretorio, numeros)
        # Crie o diretório se não existir
        if not os.path.exists(nome_diretorio):
            os.makedirs(nome_diretorio)
        # Mova o arquivo para o novo diretório
        shutil.move(os.path.join(diretorio, arquivo), os.path.join(nome_diretorio, arquivo))

print("Arquivos movidos com sucesso.")

## fix template

In [ ]:
import os 
import pyautogui
import time
import xlwings as xw
import glob
import os
from tqdm import tqdm
import openpyxl
from openpyxl.styles import Font, Alignment
import shutil
import re
import pandas as pd


# Defina o caminho do diretório
diretorio = 'testes'
arquivos = os.listdir(diretorio)
thread_limit=60
threads = []
idx=0
# Itere sobre cada arquivo no diretório
for name, group in tqdm(enumerate(arquivos), desc="Processando arquivos", total=len(arquivos)):
    thread = threading.Thread(target=salve_file, args=(os.path.join(diretorio, group),))
    thread.start()
    threads.append(thread)
    if len(threads) >= thread_limit or idx == len(arquivos) - 1:
        # Aguarde até que todas as threads terminem
        for thread in threads:
            thread.join()
        threads = []
    idx += 1